In [146]:
# Import packages
# from __future__ import print_function

import voila
import watermark
import pandas as pd
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import plotly.express as px
import plotly.graph_objects as go

In [147]:
# Original Data from https://github.com/CSSEGISandData/COVID-19
# Copyright by: "COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University"
glbconfirmed_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
glbrecovered_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")
glbdeath_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
country_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv")

In [148]:
 # Data cleaning (renaming columns to lower-case)
country_df.columns = map(str.lower, country_df.columns)
glbrecovered_df.columns = map(str.lower, glbrecovered_df.columns)
glbdeath_df.columns = map(str.lower, glbdeath_df.columns)
glbconfirmed_df.columns = map(str.lower, glbconfirmed_df.columns)

In [149]:
# Before changing column names save country_df as scatter_df for scatter plot
scatter_df = country_df.sort_values('confirmed', ascending=False)

In [150]:
# Change of 'province/state' to 'state' and 'country/region' to 'country'
glbconfirmed_df = glbconfirmed_df.rename(columns = {'province/state': 'state', 'country/region': 'country'})
glbrecovered_df = glbrecovered_df.rename(columns = {'province/state': 'state', 'country/region': 'country'})
glbdeath_df = glbdeath_df.rename(columns = {'province/state': 'state', 'country/region': 'country'})
country_df = country_df.rename(columns = {'country_region': 'country'})

In [151]:
# Change automatic float-datatype to int
country_df["confirmed"] = country_df["confirmed"].astype('Int32')
country_df["deaths"] = country_df["deaths"].astype('Int64')
country_df["active"] = country_df["active"].astype('Int64')
country_df["recovered"] = country_df["recovered"].astype('Int64')

In [152]:
# Set sorted_country_df for list-display
sorted_country_df = country_df.sort_values('confirmed', ascending=False)
sorted_country_df.drop(columns=['lat', 'long_', 'people_tested', 'people_hospitalized'], inplace = True)

In [164]:
# Define the 'higlight_col'-function to colorize certain columns
def highlight_col(x):
    p = 'background-color: grey'
    r = 'background-color: brown'
    g = 'background-color: darkgreen'
    temp_df = pd.DataFrame('', index = x.index, columns = x.columns)
    temp_df.iloc[:, 2] = p
    temp_df.iloc[:, 3] = r
    temp_df.iloc[:, 4] = g
    return temp_df

In [214]:
# Define the 'plot_cases_for_country'-function for graph-display
# Choose an input Country, iterate through glb_confirmed_df and glbdeath_df
# to display the countries total confirmed and death incidents using plotly.graphic_objects
def plot_cases_for_country(country):
    labels = ["confirmed cases", "deaths"]
    colors = ['#4C78A4', '#D62728']
    mode_size = [6, 8]
    line_size = [5, 4]
    df_list = [glbconfirmed_df, glbdeath_df]
    fig = go.Figure(layout=go.Layout(title='<span style="font-size: 24px;">Timeline</span>'))
    
    if country in np.asarray(glbconfirmed_df['country']) or str.casefold(country) == 'world':
        for i, df in enumerate(df_list): 
            if country == 'World' or country == 'world':
                x_data = np.array(list(df.iloc[:, 5:].columns))
                y_data = np.sum(np.asarray(df.iloc[:, 5:]), axis = 0)
            else:
                x_data = np.array(list(df.iloc[:, 5:].columns))
                y_data = np.sum(np.asarray(df[df['country'] == country].iloc[:, 5:]), axis = 0)

            fig.add_trace(go.Scatter(x = x_data, y = y_data, mode = 'lines', 
                                    name = labels[i],
                                    line = dict(color = colors[i], width = line_size[i]),
                                    connectgaps = True,
                                    text = "Total" + str(labels[i]) + ": " + str(y_data[-1])
                                    ))
    else:
        print("Please check the extended list below for correct spelling.") 
    if country != 'World':
        display(sorted_country_df.loc[sorted_country_df["country"] == country].style.apply(highlight_col, axis = None))
    fig.show()   

# Covid-19 Dashboard

This Dashboard offers an overview of the worldwide development of Covid-19.
The data was distributed by John Hopkins University. For further information please visit: https://coronavirus.jhu.edu/about

In [215]:
# Graph-Display: Using interact to dynamically choose a country for its confirmed cases and deaths
interact(plot_cases_for_country, country = 'World');

interactive(children=(Text(value='World', description='country'), Output()), _dom_classes=('widget-interact',)…

In [212]:
# scatter_df has different column names! plotting did not accept changed name-datatypes - is not displayed by voila
plt_scat = px.scatter(scatter_df.head(15), x='country_region', y='confirmed', size='confirmed', color='country_region', hover_name="country_region", size_max = 60)
plt_scat.update_layout(title_text = "Comparison of Confirmed Cases", title_font_size= 24, height=550, width=1000)

f = go.FigureWidget(data = plt_scat)
display(f)

FigureWidget({
    'data': [{'hovertemplate': ('<b>%{hovertext}</b><br><br>cou' ... '=%{marker.size}<extra></e…

##  Table of Worldwide Cases

In [165]:
# A list of all countries:    
sorted_country_df.style.apply(highlight_col, axis = None)

,country,last_update,confirmed,deaths,recovered,active,incident_rate,mortality_rate,uid,iso3
176,US,2020-11-19 16:25:17,11558241,250898,4350789,6956554,3508.171123,2.170728,840,USA
79,India,2020-11-19 16:25:17,8958483,131578,8383602,443303,649.163372,1.468753,356,IND
23,Brazil,2020-11-19 16:25:17,5945849,167455,5383385,395009,2797.264552,2.816335,76,BRA
62,France,2020-11-19 16:25:17,2115717,46772,150967,1917978,3241.310196,2.210693,250,FRA
141,Russia,2020-11-19 16:25:17,1998966,34525,1514945,449496,1369.769690,1.727143,643,RUS
160,Spain,2020-11-19 16:25:17,1525341,42039,150376,1332926,3262.427718,2.756039,724,ESP
180,United Kingdom,2020-11-19 16:25:17,1434004,53870,3206,1376928,2112.370615,3.756614,826,GBR
6,Argentina,2020-11-19 16:25:17,1339337,36347,1156474,146516,2963.411825,2.713805,32,ARG
85,Italy,2020-11-19 16:25:17,1308528,47870,498987,761671,2164.221697,3.658309,380,ITA
37,Colombia,2020-11-19 16:25:17,1218003,34563,1125184,58256,2393.738138,2.837678,170,COL



Please feel free to contact me for further enquiries or feedback. Link to the GitHub repository: https://github.com/JayVeezy1/CS50_final_project


The dashboard was inspired by the following tutorial: https://www.youtube.com/watch?v=FngV4VdYrkA&list=LL&index=2&ab_channel=DataSciencewithHarshit
This data set is licensed under the Creative Commons Attribution 4.0 International (CC BY 4.0) by the Johns Hopkins University on behalf of its Center for Systems Science in Engineering. Copyright Johns Hopkins University 2020. 
The correctness of this data can not by guaranteed.